In [11]:
import pandas as pd 
  
# read an excel file and convert  
# into a dataframe object 
df = pd.DataFrame(pd.read_excel("../data/outages/2023_Annual_Summary.xls")) # can also obtain thru api: https://openenergyhub.ornl.gov/explore/dataset/oe-417-annual-summaries/api/
  
df.columns = df.iloc[0]
df = df.drop(index=0)

df

,Event Month,Date Event Began,Time Event Began,Date of Restoration,Time of Restoration,Area Affected,NERC Region,Alert Criteria,Event Type,Demand Loss (MW),Number of Customers Affected
1,January,01/01/2023,00:06:00,01/01/2023,03:57:00,California: Riverside County;,WECC,Damage or destruction of its Facility that res...,Vandalism,0,0
2,January,01/01/2023,09:55:00,01/01/2023,10:30:00,California: Sacramento County;,WECC,Complete loss of monitoring or control capabil...,System Operations,0,0
3,January,01/01/2023,14:47:00,01/01/2023,16:50:00,Florida: Citrus County;,SERC,Physical attack that could potentially impact ...,Vandalism,19,3509
4,January,01/02/2023,16:12:00,01/03/2023,09:30:00,Texas: Dallas County;,TRE,Damage or destruction of its Facility that res...,Vandalism,0,0
5,January,01/02/2023,03:16:00,01/02/2023,16:46:00,New Mexico:,WECC,Physical attack that causes major interruption...,Vandalism,432,0
...,...,...,...,...,...,...,...,...,...,...,...
344,December,2023-12-25 00:00:00,08:35:00,2023-12-25 00:00:00,15:00:00,"Consolidated Edison of New York, Inc.",NPCC,Unplanned evacuation from its Bulk Electric Sy...,- Other,0,0
345,December,2023-12-27 00:00:00,14:44:00,2023-12-28 00:00:00,04:14:00,Otter Tail Power Co,MRO,Electrical System Separation (Islanding) where...,- Weather or natural disaster,14,8000
346,December,2023-12-27 00:00:00,11:11:00,2023-12-27 00:00:00,11:12:00,Pacificorp,WECC,Physical attack that could potentially impact ...,- Suspicious activity - Transmission equipment...,0,0
347,December,2023-12-30 00:00:00,21:00:00,2023-12-30 00:00:00,22:15:00,American Electric Power (Regulated Generation),RF,Physical threat to its Facility excluding weat...,- Suspicious activity,Unknown,Unknown


In [7]:
from datetime import datetime, timedelta

outages = []
dic = {}
for _, row in df.iterrows():
    # if row["Date Event Began"] == row["Date of Restoration"]:
    #     starttime = row["Time Event Began"]
    #     endtime = row["Time of Restoration"]
    #     diff = endtime - starttime
    #     lst.append(str(diff))


    try: 
        start = row["Date Event Began"] + " " + str(row["Time Event Began"])
        end = row["Date of Restoration"] + " " + str(row["Time of Restoration"])
    except:
        start = "error"
        end = "error"

    #print(type(row["Date Event Began"]), type(row["Time Event Began"]))

    if row["Number of Customers Affected"] == "Unknown":
        continue
    if row["NERC Region"] not in dic:
        dic[row["NERC Region"]] = int(row["Number of Customers Affected"])
    else:
        dic[row["NERC Region"]] += int(row["Number of Customers Affected"])

    outages.append((row["NERC Region"], start, end))

dic = {x:y for x,y in dic.items() if y!=0}
dic



# df.plot(x='Demand Loss (MW)', y='Number of Customers Affected', kind='line')
# plt.show()


{'WECC': 1089557,
 'SERC': 2000469,
 'TRE': 1606752,
 'RF': 2705961,
 'NPCC': 368284,
 'MRO': 145683,
 'MRO/RF': 585842,
 'SERC / RF': 65000,
 'RF/SERC': 118000,
 'RF ': 378,
 'SERC,MRO': 144346,
 'RE': 153000,
 nan: 778582}

In [8]:
outages

[('WECC', '01/01/2023 00:06:00', '01/01/2023 03:57:00'),
 ('WECC', '01/01/2023 09:55:00', '01/01/2023 10:30:00'),
 ('SERC', '01/01/2023 14:47:00', '01/01/2023 16:50:00'),
 ('TRE', '01/02/2023 16:12:00', '01/03/2023 09:30:00'),
 ('WECC', '01/02/2023 03:16:00', '01/02/2023 16:46:00'),
 ('SERC', '01/02/2023 18:22:00', '01/02/2023 22:27:00'),
 ('RF', '01/03/2023 11:45:00', '01/03/2023 12:00:00'),
 ('SERC', '01/05/2023 08:19:00', '01/05/2023 08:20:00'),
 ('WECC', '01/05/2023 07:30:00', '01/05/2023 09:30:00'),
 ('WECC', '01/07/2023 22:00:00', '01/08/2023 21:00:00'),
 ('TRE', '01/07/2023 23:44:00', '01/08/2023 02:29:00'),
 ('TRE', '01/08/2023 18:40:00', '01/08/2023 18:54:00'),
 ('WECC', '01/12/2023 10:00:00', 'Unkonwn Unknown'),
 ('SERC', '01/12/2023 14:00:00', '01/13/2023 03:00:00'),
 ('TRE', '01/13/2023 00:00:00', '01/13/2023 01:00:00'),
 ('RF', '01/20/2023 10:45:00', 'Unknown Unknown'),
 ('NPCC', '01/23/2023 07:05:00', '01/24/2023 17:17:00'),
 ('SERC', '01/24/2023 14:00:00', '01/24/2023 15

In [10]:
# datetime tutorial

from datetime import datetime

datetime_str = '13:55:26'

datetime_object = datetime.strptime(datetime_str, '%H:%M:%S')
time_object = datetime.strptime(datetime_str, '%H:%M:%S').time() #stripping date (.date() strips the time)

print(type(datetime_object), type(time_object))
print("date and time:", datetime_object)
print("only time:", time_object)

<class 'datetime.datetime'> <class 'datetime.time'>
date and time: 1900-01-01 13:55:26
only time: 13:55:26
